# CSI4142 Project Phase 2

Group 45 Members

|Name|Student No.|Email
|----|-----------|-----
|Hongyi Lin| 300053082| hlin087@uottawa.ca
|Rodger Retanal| 300052309| rreta014@uottawa.ca
|Hao Jing| 300071593|hjing098@uottawa.ca

## Dependencies
Use the following commands to install required packages for this notebook.
```bash
# pip
pip install pandas
pip install psycopg2

# conda
conda install pandas
conda install psycopg2
```


In [6]:
from database_utils import db_connect

In [7]:
# try to establish the connection
db_conn, db_cur = db_connect()
# verify the connection
db_cur.execute('SELECT version()')
db_version = db_cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
('PostgreSQL 11.11 on amd64-portbld-freebsd11.4, compiled by FreeBSD clang version 10.0.0 (git@github.com:llvm/llvm-project.git llvmorg-10.0.0-0-gd32170dbd5b), 64-bit',)


In [8]:
# create the tables by executing init.sql
db_cur.execute(open("init.sql", "r").read())
db_conn.commit()

In [9]:
# close the cursor
if db_cur is not None:
    db_cur.close()

# close the connection
if db_conn is not None:
    db_conn.close()